In [1]:
%autosave 0

Autosave disabled


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 30.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [1]:
import os
import pandas as pd
import numpy as np
import re
import collections
import pickle
from ast import literal_eval
import random
import scipy
import math
import datetime
import subprocess

from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

#Functions

In [2]:
# arrange sequence of tour

def arrange_tour(solver_tour, start_end_points):

    #Get the start and end point of the specific district of the instance
    start_PP, end_PP = start_end_points 
    arranged_tour = 0

    #FUNCTION TO REARRANGE THE LIST WITH CASE OF SAME START AND END POSTPOINT
    def rearrange_list(numbers, specific_value):
        # Find the index of the specific value in the list
        index = numbers.index(specific_value)
        
        # Check if the chosen element is repeated twice consecutively
        if numbers[index] == numbers[index + 1]:
            # Split the list into two parts based on the index
            part1 = numbers[:index]
            part2 = numbers[index + 1:]
            
            # Rearrange the list by concatenating the parts and adding the specific value at the beginning and end
            rearranged_list = part2 + part1 + [specific_value]
        else:
            # Split the list into two parts based on the index
            part1 = numbers[:index]
            part2 = numbers[index:]
            
            # Rearrange the list by concatenating the parts in reverse order
            rearranged_list = part2 + part1
        
        return rearranged_list
    
    
    #FUNCTION TO REARRANGE THE LIST WITH CASE OF DIFFERENT START AND END POSTPOINT
    def rearrange_circular_tour(tour, start_point, end_point):
    # Find the index of the start point in the tour list
        start_index = tour.index(start_point)
            
        # Find the index of the end point in the tour list
        end_index = tour.index(end_point)
                
        # Check if the start and end points are already at the first and last positions
        if start_index == 0 and end_index == len(tour) - 1:
            return tour
                
        # Split the tour list into two parts based on the start and end indices
        if start_index < end_index:
            part1 = tour[0:start_index + 1]
            part1.reverse()

            part2 = tour[start_index + 1:]
            part2.reverse()
                
        else:
            part1 = tour[start_index:] + tour[:end_index+1]
            part2 = tour[end_index+1:start_index]
                
        # Rearrange the tour list by concatenating the parts
        rearranged_tour = part1 + part2
        
        return rearranged_tour
    


    #Case of start and end point equal
    if start_PP == end_PP:
        #If the start and end point are already in correct positions return list as it is                                                                              
        if start_PP == solver_tour[0] and end_PP == solver_tour[len(solver_tour)-1]:                    
            arranged_tour = solver_tour
        else: 
            #Check if start or end point (doesn't matter they have the same value) appears less than two times on the list
            if list.count(solver_tour, start_PP) < 2:
                #If if is not repeated. Check if at least start_point is in first_position.                                                   
                if start_PP == solver_tour[0]: 
                    #If start_point is in first position just append end point at the end of list.                                                         
                    arranged_tour = solver_tour + [end_PP]
                else: 
                    #Rearrange the tour first with start_PP at the beginning of the list before appending end_PP
                    arranged_tour = rearrange_list(solver_tour, start_PP)
                    arranged_tour = arranged_tour + [end_PP]               
            else: 
                #Rearrange the tour in the correct order
                arranged_tour = rearrange_list(solver_tour, start_PP)
    #Case of start and end point different
    else:
       arranged_tour = rearrange_circular_tour(tour = solver_tour, start_point = start_PP, end_point = end_PP)
    
    return arranged_tour

In [3]:
#FUNCTION TO GET SUBSET OF A COMPLETE TOUR AND ITS LENGTH KEEPING SEQUENCE BASED ON INSTANCES VOLUMES

#INPUTS:    Complete Tour (DP or Concord), Complete Tour info (DP or Concord), 
#           Distance Matrix for the instance (generated from instance generation function), Mapping of PostPoints Needed (Dictionary obtained from instance generation function)
#OUTPUTS:   Tour Sequence Fixed, Tour Length, Difference between New Tour and Complete Tour



def new_tour_sequence_fixed(complete_tour, dm_instance, mapping_pp_needed):
    #get the list of only the Postpoint needed for the specific instance
    list_pp_needed = list(mapping_pp_needed.values())

    #Keep from complete_tour only the needed PostPoints according to the list of PostPoints needed mantaining sequence
    tour_seq_fixed = [x for x in complete_tour if x in list_pp_needed]

    #Generate a dataframe from the distance matrix with the correct row-column combination based on PostPoints required
    df_dm_instance = pd.DataFrame(dm_instance)
    df_dm_instance = (df_dm_instance.rename(columns= mapping_pp_needed)).rename(index = mapping_pp_needed)

    #FUNCTION to compute length of new tour (Adapting using logic of function already created for default_route calculation: get_tour_length())
    
    def get_seq_tour_length(sequence_list, distance_df):
        tour_length = 0 
        for i in range(len(sequence_list)-1):
            # Specify the specific values for row and column in the dataframe
            PostPoint_predecesor = sequence_list[i]
            PostPoint_succesor = sequence_list[i+1]

            # Get the distance value needed based on the Post Point combination or raise error
            try:
                distance_value = distance_df[PostPoint_predecesor][PostPoint_succesor]
            except: 
                print('ERROR: No distance for PostPoint combination')

            # Aggreate sum value until end of loop to compute total tour length
            tour_length += distance_value
            
        return round(tour_length,2)
    
    #Calculate tour length of new tour which skips PostPoints
    tour_seq_fixed_length = get_seq_tour_length(tour_seq_fixed, df_dm_instance)
    

    return tour_seq_fixed, tour_seq_fixed_length

In [4]:
# route similarity

def find_similarity_route(Tour1, Tour2):
    predecessors1 = {Tour1[i]: Tour1[i-1] for i in range(1, len(Tour1))}
    predecessors2 = {Tour2[i]: Tour2[i-1] for i in range(1, len(Tour2))}
    num_changed_predecessors = 0

    for node, pred2 in predecessors2.items():
        pred1 = predecessors1[node]
        if pred1 != pred2:
            num_changed_predecessors += 1
    percent_change = num_changed_predecessors / (len(Tour1)-1) * 100
    sim_pct = 100 - percent_change

    return num_changed_predecessors, sim_pct

In [5]:
# heuristic google OR algo

class TSP:
    def __init__(self, distance_matrix):
        #read distance matrix --> list
        self.dist_mat = distance_matrix
        self.dist_mat = self.round_up(self.dist_mat)
        # Create data model
        self.data = self.create_data_model()
        # Create routing index manager
        self.manager = pywrapcp.RoutingIndexManager(len(self.data['distance_matrix']),
                                            self.data['num_vehicles'], self.data['depot'])
        # Create Routing Model
        self.routing = pywrapcp.RoutingModel(self.manager)
        # Define cost of each arc
        self.transit_callback_index = self.routing.RegisterTransitCallback(self.distance_callback)
        self.routing.SetArcCostEvaluatorOfAllVehicles(self.transit_callback_index)
    
    def round_up(self , lst):
        '''
        input : list of lists having non integral values
        output : list of lists having integral values.
        Function to round up numbers and return as integers
        '''
        rounded_lst = []
        for inner_lst in lst:
            rounded_inner_lst = []
            for num in inner_lst:
                rounded_num = int(round(num))
                rounded_inner_lst.append(rounded_num)
            rounded_lst.append(rounded_inner_lst)
        return rounded_lst        


    def create_data_model(self):
        # Stores the data for the problem
        data = {}
        data['distance_matrix'] = self.dist_mat 
        data['num_vehicles'] = 1
        data['depot'] = 0
        return data

    def distance_callback(self, from_index, to_index):
        # Returns the distance between the two nodes
        from_node = self.manager.IndexToNode(from_index)
        to_node = self.manager.IndexToNode(to_index)
        return self.data['distance_matrix'][from_node][to_node]

    def solve(self):
        # Setting first solution heuristic
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
        # Solve the problem
        solution = self.routing.SolveWithParameters(search_parameters)
        if solution:
            # Get the route as an array
            route = []
            index = self.routing.Start(0)
            while not self.routing.IsEnd(index):
                node = self.manager.IndexToNode(index)
                route.append(node)
                index = solution.Value(self.routing.NextVar(index))
            # Get the objective value
            obj_value = solution.ObjectiveValue()
            # route = self.map_nodes(route)
            return route, obj_value

In [6]:
def map_actual_route(route, mapping):
  tour_mapped = [mapping[a] for a in route]
  return tour_mapped

In [7]:
# heuristic google OR algo
# solver using global cheapest arc

class TSP_v2:
    def __init__(self, distance_matrix):
        #read distance matrix --> list
        self.dist_mat = distance_matrix
        self.dist_mat = self.round_up(self.dist_mat)
        # Create data model
        self.data = self.create_data_model()
        # Create routing index manager
        self.manager = pywrapcp.RoutingIndexManager(len(self.data['distance_matrix']),
                                            self.data['num_vehicles'], self.data['depot'])
        # Create Routing Model
        self.routing = pywrapcp.RoutingModel(self.manager)
        # Define cost of each arc
        self.transit_callback_index = self.routing.RegisterTransitCallback(self.distance_callback)
        self.routing.SetArcCostEvaluatorOfAllVehicles(self.transit_callback_index)
    
    def round_up(self , lst):
        '''
        input : list of lists having non integral values
        output : list of lists having integral values.
        Function to round up numbers and return as integers
        '''
        rounded_lst = []
        for inner_lst in lst:
            rounded_inner_lst = []
            for num in inner_lst:
                rounded_num = int(round(num))
                rounded_inner_lst.append(rounded_num)
            rounded_lst.append(rounded_inner_lst)
        return rounded_lst        


    def create_data_model(self):
        # Stores the data for the problem
        data = {}
        data['distance_matrix'] = self.dist_mat 
        data['num_vehicles'] = 1
        data['depot'] = 0
        return data

    def distance_callback(self, from_index, to_index):
        # Returns the distance between the two nodes
        from_node = self.manager.IndexToNode(from_index)
        to_node = self.manager.IndexToNode(to_index)
        return self.data['distance_matrix'][from_node][to_node]

    def solve(self):
        # Setting first solution heuristic
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.GLOBAL_CHEAPEST_ARC
        # Solve the problem
        solution = self.routing.SolveWithParameters(search_parameters)
        if solution:
            # Get the route as an array
            route = []
            index = self.routing.Start(0)
            while not self.routing.IsEnd(index):
                node = self.manager.IndexToNode(index)
                route.append(node)
                index = solution.Value(self.routing.NextVar(index))
            # Get the objective value
            obj_value = solution.ObjectiveValue()
            # route = self.map_nodes(route)
            return route, obj_value

#Read necessary files

In [8]:
# upload files
all_original = pickle.load(open("/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/Results/all_original_complete.p", "rb"))
warmsen_instances = pickle.load(open("/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/Results/warmsen_instances.p", "rb"))
all_default_tours = pickle.load(open("/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Coding/Results/all_default_tours.p", "rb"))

In [9]:
all_original['Warmsen']['31606-14'].keys()

dict_keys(['start_end_points', 'dm', 'map'])

#Depo Check

In [10]:
all_original['Warmsen']['31606-14']['start_end_points']

[0, 4409]

In [11]:
all_original['Warmsen']['31606-14']['start_end_points'][0] == all_original['Warmsen']['31606-14']['start_end_points'][1]

False

In [12]:
diff_depo_all = {}

# iterate through all regions
for region in all_original.keys():
  diff_depo_region = {}
  # iterate through districts in region
  for district in all_original[region].keys():
    # check whether start and end point is the same
    same_depo = all_original[region][district]['start_end_points'][0] == all_original[region][district]['start_end_points'][1]
    print(region, ':', district, ': depos', all_original[region][district]['start_end_points'][0],
          all_original[region][district]['start_end_points'][1],
          'same depo -->', same_depo)
    # store district data
    if same_depo == False:
      diff_depo_region[district] = all_original[region][district]
  # store region data
  diff_depo_all[region] = diff_depo_region

Warmsen : 31606-13 : depos 4 4409 same depo --> False
Warmsen : 31606-11 : depos 4 3 same depo --> False
Warmsen : 31600-04 : depos 0 0 same depo --> True
Warmsen : 31600-03 : depos 0 2 same depo --> False
Warmsen : 31600-02 : depos 0 0 same depo --> True
Warmsen : 31603-07 : depos 0 4194 same depo --> False
Warmsen : 31603-08 : depos 4 4194 same depo --> False
Warmsen : 31604-10 : depos 0 3 same depo --> False
Warmsen : 31603-05 : depos 0 3 same depo --> False
Warmsen : 31603-06 : depos 0 3 same depo --> False
Warmsen : 31606-12 : depos 0 4194 same depo --> False
Warmsen : 31606-14 : depos 0 4409 same depo --> False
Warmsen : 31604-09 : depos 1159 3 same depo --> False
Warmsen : 31600-01 : depos 0 0 same depo --> True
Uerze : 31311-31 : depos 0 2 same depo --> False
Uerze : 31311-07 : depos 0 1 same depo --> False
Uerze : 31311-04 : depos 0 4 same depo --> False
Uerze : 31311-01 : depos 0 4 same depo --> False
Uerze : 31311-25 : depos 0 2 same depo --> False
Uerze : 31311-22 : depos 0

In [13]:
diff_depo_all['Hannover 92'].keys()

dict_keys(['30459-35'])

In [14]:
diff_depo_all['Hannover 92']['30459-35'].keys()

dict_keys(['start_end_points', 'dm', 'map'])

In [15]:
diff_depo_all['Hannover 92']['30459-35']['start_end_points']

[0, 5267]

# Google OR --> Path_Cheapest_Arc

In [16]:
%%time

# iterate through all regions
for region in diff_depo_all.keys():
  # iterate through all districts
  for district in diff_depo_all[region].keys():
    start = datetime.datetime.now()
    # solve TSP
    tsp = TSP(diff_depo_all[region][district]['dm'])
    # store TSP results
    solver_info = {}
    solver_info['tour'], solver_info['cost'] = tsp.solve()
    solver_info['tour'] = map_actual_route(solver_info['tour'], diff_depo_all[region][district]['map'])
    diff_depo_all[region][district]['path_cheapest_arc'] = solver_info
    # log to check whether start and end points are adjacent
    print(region, ';', district, ';', 'depos -->', diff_depo_all[region][district]['start_end_points'],
          '\n; start and end segments after using solver -->', solver_info['tour'][0:5], solver_info['tour'][-5:],
          ';time needed :', datetime.datetime.now() - start)


Warmsen ; 31606-13 ; depos --> [4, 4409] 
; start and end segments after using solver --> [4, 4409, 4197, 4195, 4269] [4270, 4179, 4407, 4408, 4200] ;time needed : 0:00:06.254171
Warmsen ; 31606-11 ; depos --> [4, 3] 
; start and end segments after using solver --> [3, 4, 4490, 4491, 4493] [4610, 4609, 4608, 4607, 4489] ;time needed : 0:00:04.799789
Warmsen ; 31600-03 ; depos --> [0, 2] 
; start and end segments after using solver --> [0, 2, 8, 767, 768] [906, 905, 904, 903, 902] ;time needed : 0:00:07.297881
Warmsen ; 31603-07 ; depos --> [0, 4194] 
; start and end segments after using solver --> [0, 4194, 3003, 2999, 2998] [2841, 2844, 2845, 2847, 2849] ;time needed : 0:00:06.156818
Warmsen ; 31603-08 ; depos --> [4, 4194] 
; start and end segments after using solver --> [4, 3145, 3148, 3149, 3146] [3278, 3266, 3265, 3267, 4194] ;time needed : 0:00:07.984165
Warmsen ; 31604-10 ; depos --> [0, 3] 
; start and end segments after using solver --> [0, 3953, 3944, 3945, 3947] [4148, 4149,

# Google OR --> Global_Cheapest_Arc

In [17]:
%%time

# iterate through all regions
for region in diff_depo_all.keys():
  # iterate through all districts
  for district in diff_depo_all[region].keys():
    start = datetime.datetime.now()
    # solve TSP
    tsp = TSP_v2(diff_depo_all[region][district]['dm'])
    # store TSP results
    solver_info = {}
    solver_info['tour'], solver_info['cost'] = tsp.solve()
    solver_info['tour'] = map_actual_route(solver_info['tour'], diff_depo_all[region][district]['map'])
    diff_depo_all[region][district]['global_cheapest_arc'] = solver_info
    # log to check whether start and end points are adjacent
    print(region, ';', district, ';', 'depos -->', diff_depo_all[region][district]['start_end_points'],
          '\n; start and end segments after using solver -->', solver_info['tour'][0:5], solver_info['tour'][-5:],
          ';time needed :', datetime.datetime.now() - start)


Warmsen ; 31606-13 ; depos --> [4, 4409] 
; start and end segments after using solver --> [4, 4195, 4201, 4194, 4196] [4172, 4178, 4179, 4408, 4409] ;time needed : 0:00:04.233139
Warmsen ; 31606-11 ; depos --> [4, 3] 
; start and end segments after using solver --> [3, 4, 4490, 4606, 4611] [4574, 4575, 4576, 4492, 4489] ;time needed : 0:00:07.676156
Warmsen ; 31600-03 ; depos --> [0, 2] 
; start and end segments after using solver --> [0, 885, 877, 876, 901] [937, 936, 938, 839, 2] ;time needed : 0:00:09.836984
Warmsen ; 31603-07 ; depos --> [0, 4194] 
; start and end segments after using solver --> [0, 4194, 3003, 2999, 2998] [2849, 2850, 2790, 2792, 2791] ;time needed : 0:00:10.837166
Warmsen ; 31603-08 ; depos --> [4, 4194] 
; start and end segments after using solver --> [4, 4194, 3145, 3148, 3149] [3051, 3050, 3048, 3349, 3348] ;time needed : 0:00:09.540108
Warmsen ; 31604-10 ; depos --> [0, 3] 
; start and end segments after using solver --> [0, 3842, 3829, 3828, 3841] [4152, 414

#Comparison between the solution strategies

In [30]:
%%time

# iterate through all regions
for region in diff_depo_all.keys():
  # iterate through all districts
  for district in diff_depo_all[region].keys():
    strategy_1 = diff_depo_all[region][district]['path_cheapest_arc']
    strategy_2 = diff_depo_all[region][district]['global_cheapest_arc']
    # calculate similarity of tours
    tour_sim = round(find_similarity_route(strategy_1['tour'], strategy_2['tour'])[1], 2)
    # calculate  ratio of using strategy 2 compared to strategy 1
    strat_ratio = round(strategy_2['cost']/strategy_1['cost'], 2)
    # log to check
    print(region, ';', district, '; tour similarity', tour_sim, '%',
          '; ratio using global_cheapest_arc vs path_cheapest_arc', strat_ratio)


Warmsen ; 31606-13 ; tour similarity 53.11 % ; ratio using global_cheapest_arc vs path_cheapest_arc 1.0
Warmsen ; 31606-11 ; tour similarity 52.87 % ; ratio using global_cheapest_arc vs path_cheapest_arc 0.98
Warmsen ; 31600-03 ; tour similarity 59.3 % ; ratio using global_cheapest_arc vs path_cheapest_arc 0.99
Warmsen ; 31603-07 ; tour similarity 60.86 % ; ratio using global_cheapest_arc vs path_cheapest_arc 1.0
Warmsen ; 31603-08 ; tour similarity 53.76 % ; ratio using global_cheapest_arc vs path_cheapest_arc 1.0
Warmsen ; 31604-10 ; tour similarity 56.46 % ; ratio using global_cheapest_arc vs path_cheapest_arc 0.98
Warmsen ; 31603-05 ; tour similarity 44.27 % ; ratio using global_cheapest_arc vs path_cheapest_arc 1.0
Warmsen ; 31603-06 ; tour similarity 61.47 % ; ratio using global_cheapest_arc vs path_cheapest_arc 1.01
Warmsen ; 31606-12 ; tour similarity 54.89 % ; ratio using global_cheapest_arc vs path_cheapest_arc 1.0
Warmsen ; 31606-14 ; tour similarity 58.06 % ; ratio using gl